# Python CLI

Los objetivos de aprendizaje son:

1. Qué es una CLI
    - Estructura
2. CLIs Básicas con Python
    - sys.argv
    - argparse
    - Añadir Argumentos y Opciones
 
    
## ¿Qué es una CLI?

Las interfaces de línea de comandos permiten interactuar **programaticamente** con una aplicación o programa a través de la línea de comandos, terminal o consola del sistema operativo.

Por ejemplo, el comando `ls` (`dir` Windows) lista las carpetas y directorios de una ruta. Por defecto el el actual directorio.

In [1]:
!ls version_2/

__pycache__   test_utils.py utils.py


Supongamos que deseamos información más completa sobre su directorio y su contenido, el comando `ls` tiene una un conjunto útiles de opciones que podemos usar para personalizar el comportamiento del comando.

Por ejemplo:

In [2]:
!ls -l version_2/

total 16
drwxr-xr-x  4 heber.trujillo  staff  128 Jan 14 18:10 __pycache__
-rw-r--r--@ 1 heber.trujillo  staff  407 Jan 13 11:43 test_utils.py
-rw-r--r--@ 1 heber.trujillo  staff  164 Jan 13 11:43 utils.py


La salida de `ls` muestra mucha más información sobre los archivos e.g. permisos, el propietario, el grupo, la fecha y el tamaño. También muestra el espacio total que utilizan estos archivos en el disco.

Este cabio resulta del uso de la opción -l

### Estructura

Un comando puede tener los siguientes elementos:

- **Comando**: Un programa que se ejecuta en la línea de comando. Por lo general, se identifica con el nombre del programa o rutina subyacente, e.g. `poetry`.
<br>

- **Argumento**: Pieza de información obligatoria u opcional que utiliza un comando para realizar una acción prevista. Los comandos suelen aceptar uno o varios argumentos, que puede proporcionar como una lista separada por espacios en blanco o por comas en la línea de comandos.
<br>

- **Opciones**: Un argumento opcional que modifica el comportamiento de un comando. Las opciones se pasan a los comandos usando un nombre específico, como `-l` en el ejemplo anterior.
<br>

- **Parámetro**: Un argumento que utiliza una opción para realizar la operación o acción prevista.
<br>

- **Subcomando**: Un nombre predefinido que se puede pasar a una aplicación para ejecutar una acción específica.


Por ejemplo:

```shell
poetry new cac-poetry --name cac_con_poetry
```

- poetry: Comando
- new: Subcomando
- cac-poetry: Argumento
- --name: Opción
- cac_con_poetry: Parámetro


## CLIs Básicas con Python

Python viene con un par de herramientas que podemos usar para escribir CLIs para nuestros scripts o aplicaciones. 

### `sys.argv`

El  atributo `argv` del módulo `sys` almacena dentro de una lista los argumentos que pasa a un programa en la línea de comandos. El primer item de `argv` es el nombre del programa.

Supongamos que queremos escribir un programa para enumerar todos los archivos en un directorio determinado, similar a lo que hace `ls`.


```python
# ls_argv.py 

import sys
from pathlib import Path

print(sys.argv)

if (args_count := len(sys.argv)) > 2:
    print(f"Se esperaba un argumento, no {args_count - 1}")
    raise SystemExit(2)
elif args_count < 2:
    print("Se debe indicar el directorio")
    raise SystemExit(2)

target_dir: Path = Path(sys.argv[1])

if not target_dir.is_dir():
    print(f"El directorio {sys.argv[1]} no existe")
    raise SystemExit(1)

for entry in target_dir.iterdir():
    print(entry.name)
```

In [7]:
!python ls_argv.py version_2

El contenido de argv es: ['ls_argv.py', 'version_2']
test_utils.py
.pytest_cache
__pycache__
utils.py


> **Nota**: Se trata del [operador walrus](https://docs.python.org/3/reference/expressions.html#assignment-expressions) `:=` asigna un valor a una variable lo evalúa como expresión booleana.

Para ejecutar el escript usamos el siguiente comando:
``` shell
python ls_argv.py version_2
```

Aunque el programa funciona, analizar los argumentos de la línea de comandos manualmente usando el atributo `sys.argv` no es una solución escalable para aplicaciones CLI más complejas.


## `argparse`

Una forma mucho más conveniente de crear aplicaciones CLI es usar el módulo `argparse`, que viene en la biblioteca estándar.

Este módulo se lanzó por primera vez en Python 3.2 con PEP 389 y es una forma rápida de crear aplicaciones sin instalar paquetes de terceros, como [Typer](https://typer.tiangolo.com/) o [Click](https://click.palletsprojects.com/en/8.1.x/).


Para usar `argparse` se deben seguir los siguientes pasos:

1. Importar `argparse`.
<br>

2. Crear una instancia de la clase `ArgumentParser`.
<br>

3. Añadir argumentos y opciones a la instancia de `ArgumentParser` usando el método `.add_argument()`.
<br>

4. LLamar al método `.parse_args()` para obtener los argumentos en forma de un objeto `Namespace`.

Por ejemplo:

```python 
# ls_argparse.py

import argparse
from pathlib import Path

parser = argparse.ArgumentParser()

parser.add_argument("path")

args = parser.parse_args()

target_dir = Path(args.path)

if not target_dir.exists():
    print(f"El directorio {sys.argv[1]} no existe")
    raise SystemExit(1)

for entry in target_dir.iterdir():
    print(entry.name)
```

El código ha cambiado significativamente. La diferencia más notable con la versión anterior es que los *conditional statements* para verificar los argumentos proporcionados por el usuario ya no están porque `argparse` verifica automáticamente la presencia de argumentos.

````
python ls_argparse.py version_2 
````


Si ejecutamos el siguiente comando:

```
python ls_argparse.py
```

Veremos una nueva característica, ahora nuestro programa acepta una opción -h opcional. Si ejecutamos:

````
python ls_argparse.py -h
````

Veremos un mensaje de ayuda con instrucciones de uso. 


El módulo `argparse` reconoce dos tipos diferentes de argumentos de línea de comandos:

- **Argumentos posicionales**: Que definimos como como argumentos previamente.
<br>

- **Argumentos opcionales**: Que definimos como opciones, a.k.a. flags o switches.


En `ls_argparse.py` `path` es un argumento posicional.

### Añadir Argumentos y Opciones

El método `.add_argument()` de la clase `ArgumentParser` se puede usar para añadir ambos. 

El primer Argumento del método [`.add_argument()`](https://docs.python.org/3/library/argparse.html#argparse.ArgumentParser.add_argument) establece la diferencia entre argumentos y opciones. 

Cuando se llama a `parse_args()`, los argumentos opcionales se identificarán con el prefijo `-` o `--`.

Veamos un ejemplo

``` python

# ls_argparse_2.py

import argparse
import datetime
from pathlib import Path, PosixPath

parser = argparse.ArgumentParser()

parser.add_argument("path")

parser.add_argument("-l", "--long", action="store_true")

args = parser.parse_args()

target_dir = Path(args.path)

if not target_dir.exists():
    print(f"El directorio {sys.argv[1]} no existe")
    raise SystemExit(1)

def build_output(entry: PosixPath, long: bool = False):
    if long:
        size = entry.stat().st_size
        date = datetime.datetime.fromtimestamp(
            entry.stat().st_mtime).strftime(
            "%b %d %H:%M:%S"
        )
        return f"{size:>6d} {date} {entry.name}"
    return entry.name

for entry in target_dir.iterdir():
    print(build_output(entry, long=args.long))
```

Ejecutemos el comando:

````
python ls_argparse_2.py version_2 -l
````

Para el argumento opcional `-l` hemos establecido la acción `"store_true"`, significa que esta opción almacenará un valor booleano `True` si proporciona la opción en la línea de comando y `False` en caso contrario.


### Parsing Command-Line.

En este contexto `Parsing` hace referencia a analizar y almacenar los argumentos y opciones dentro de un `Namespace`, en el ejemplo anterior lo llamamos `args`.

```` python
args = parser.parse_args()
````

El método `.parse_args()` convierte las strings dentro de la lista de argumentos en objetos y y los asigna a atributos del `Namespace`:

In [8]:
from argparse import ArgumentParser
parser = ArgumentParser()
parser.add_argument("asegurado")
parser.add_argument("-v", "--vigente", action="store_true")
parser.add_argument("-c", "--cobertura", type=str, action="store", default="RC")

_StoreAction(option_strings=['-c', '--cobertura'], dest='cobertura', nargs=None, const=None, default='RC', type=<class 'str'>, choices=None, required=False, help=None, metavar=None)

In [9]:
args = parser.parse_args(args=["heber"])
args

Namespace(asegurado='heber', vigente=False, cobertura='RC')

In [10]:
args = parser.parse_args(args=["heber", "-v"])
args

Namespace(asegurado='heber', vigente=True, cobertura='RC')

In [11]:
from dataclasses import dataclass

@dataclass
class Parametros:
    asegurado: str
    vigente: bool
    cobertura: str

args = parser.parse_args(args=["heber", "-v"], namespace=Parametros)
args

__main__.Parametros

In [12]:
args.asegurado

'heber'